## A11535519 - Justin Laughlin
### MAE 290A: Homework 1 (10/19/17)
### Problem 3

In [1]:
# Import necessary packages & configure settings
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import timeit

%matplotlib inline

### Governing equations:

<img src="hw1_3.png">

### Known parameters

In [2]:
m     = 1000 # mass of floors [kg]
alpha = np.sqrt(2)/2 # sin(pi/4)=cos(pi/4)
g     = 9.8 # gravitational acceleration [m/s^2]
d1 = d3 = d5 = d7 = 1000 # Tension [N]

### Rewriting equations in matrix form
**NOTE: my matrix may look different as the order of equations has been rearranged slightly (moves from bottom to top for "vertical forces" then bottom to top for "horizontal forces"**

$$\begin{pmatrix}
0 & 1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \alpha & 0 & 0\\
1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\alpha & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \alpha & 0\\
0 & 0 & 1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\alpha & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \alpha\\
0 & 0 & 0 & 0 & 1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\alpha & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\alpha\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -\alpha & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & \alpha & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -\alpha & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & \alpha & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -\alpha\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & \alpha & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & \alpha\\
\end{pmatrix}
\begin{pmatrix}
p_1 \\
p_2 \\
p_3 \\
p_4 \\
p_5 \\
p_6 \\
p_7 \\
p_8 \\
f_1 \\
f_2 \\
f_3 \\
f_4 \\
d_2 \\
d_4 \\
d_6 \\
d_8 \\
\end{pmatrix} \\ =
\begin{pmatrix}
mg/2 + \alpha d_1\\
mg/2 - \alpha d_3\\
mg/2 + \alpha d_3\\
mg/2 - \alpha d_5\\
mg/2 + \alpha d_5\\
mg/2 - \alpha d_7\\
mg/2 + \alpha d_7\\
mg/2 \\
\alpha d_1 \\
-\alpha d_3 \\
\alpha d_3 \\
-\alpha d_5 \\
\alpha d_5 \\
-\alpha d_7 \\
\alpha d_7 \\
0 \\
\end{pmatrix}$$

First, solve for $\mathbf{x} = \begin{pmatrix}
p_1, &
p_2, &
p_3, &
p_4, &
p_5, &
p_6, &
p_7, &
p_8, &
f_1, &
f_2, &
f_3, &
f_4, &
d_2, &
d_4, &
d_6, &
d_8
\end{pmatrix}^T$
using $\text{LU}$ decomposition

In [3]:
# Setup our LHS matrix, A. Also, setup our RHS vector, f
def initAf(m,d1,d3,d5,d7):
    A = np.array(([0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, alpha, 0, 0],
    [1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -alpha, 0, 0, 0],
    [0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, alpha, 0],
    [0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -alpha, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, alpha],
    [0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, -alpha, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -alpha],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -alpha, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, alpha, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -alpha, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, alpha, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -alpha],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, alpha, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, alpha]))

    f = np.array(([m*g/2 + alpha*d1, 
                   m*g/2 - alpha*d3, 
                   m*g/2 + alpha*d3, 
                   m*g/2 - alpha*d5, 
                   m*g/2 + alpha*d5, 
                   m*g/2 - alpha*d7, 
                   m*g/2 + alpha*d7, 
                   m*g/2,
                   alpha*d1,
                   -alpha*d3,
                   alpha*d3,
                   -alpha*d5,
                   alpha*d5,
                   -alpha*d7,
                   alpha*d7,
                   0]))
    return A,f

In [4]:
A,f = initAf(m,d1,d3,d5,d7)

### $\mathbf{LU}$ decomposition

Since $\mathbf{A}=\mathbf{P\cdot L\cdot U}$ and we are trying to solve $\mathbf{A}\cdot \mathbf{x} = \mathbf{f}$ we can write
$$\left( \mathbf{P\cdot L\cdot U} \right) \cdot \mathbf{x} = \mathbf{f}$$
Additionally, $\mathbf{P}$ is an orthogonal matrix (we will demonstrate this below by showing $\text{det}(\mathbf{P})=\pm 1$), which means that $P^T = P^{-1}$. Therefore we can write
$$\left( \mathbf{L\cdot U} \right) \cdot \mathbf{x} = \mathbf{P^T} \cdot \mathbf{f}$$
Let us define a vector $\mathbf{y} = \mathbf{U} \cdot \mathbf{x}$. Now we have two simple problems to solve which can be done with forward and backward substitution, respectively.

$$\mathbf{L} \cdot \mathbf{y} = \mathbf{P^T} \cdot \mathbf{f}$$
$$\mathbf{U} \cdot \mathbf{x} = \mathbf{y}$$

First, define functions for forward and backward substitution which are required once $\mathbf{A}$ has been decomposed into lower and upper triangular matrices. Also, for convenience, we will define a function LUsolve(A,f) which automatically applies all necessary functions to solve for $\mathbf{x}$:

In [5]:
# Forward substitution
# Solves Ly=f when L is lower diagonal
def fsub(L,f):
    # Initialize
    N = len(f)
    y = np.ndarray((N,), dtype=np.double)
    y[0] = f[0]/L[0,0]
    
    # Forward sub
    for j in np.arange(1,N):
        y[j] = (f[j] - np.dot(L[j,0:j],y[0:j]))/L[j,j]
    
    return y

# Backward substitution
# Solves Ux=y when U is upper diagonal
def bsub(U,y):
        # Initialize
    N = len(y)
    x = np.ndarray((N,), dtype=np.double)
    x[-1] = y[-1]/U[N-1,N-1]
    
    # Backward sub
    for j in np.arange(N-2,-1,-1):
        x[j] = (y[j] - np.dot(U[j,(j+1):],x[(j+1):]))/U[j,j]
    
    return x

# Solve Ax=f using LU decomposition
def LUsolve(A,f):
    # Perform LU decomposition. In scipy's implementation A = PLU
    P, L, U = linalg.lu(A)
    F = np.dot(P.T,f)
    y = fsub(L,F)
    x = bsub(U,y)
    
    return x

In [6]:
# This wrapper allows functions such as LUsolve() to be timed by timeit.timeit() so we can compare performance with Gauss Jordan
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

In [7]:
# Confirm that P is orthogonal (det(P)= +-1) allowing P^T = P^{-1}
P, L, U = linalg.lu(A)
print("det(P) =",linalg.det(P))

det(P) = 1.0


### Gauss-Jordan Elimination

In [8]:
# Returns the inverse of matrix A. Uses full pivoting.
def gaussJordan(Anew,bnew):
    # Initialize
    n = len(bnew)
    indxc = [None]*n
    indxr = [None]*n
    ipiv  = np.zeros((n,1), dtype=np.int64)
    
    # Main loop
    for i in np.arange(n):
        big = 0.0;
        for j in np.arange(n):
            if ipiv[j] != 1:
                for k in np.arange(n):
                    if ipiv[k] == 0:
                        if np.abs(Anew[j,k]) >= big:
                            big = abs(Anew[j,k])
                            irow = j
                            icol = k
        ipiv[icol] += 1
        if irow != icol:
            for l in np.arange(n):
                Anew[irow,l], Anew[icol,l] = Anew[icol,l], Anew[irow,l]
            bnew[irow], bnew[icol] = bnew[icol], bnew[irow]
        indxr[i] = irow
        indxc[i] = icol
        if Anew[icol,icol] == 0.0:
            print('gaussJordan: Singular Matrix!')
        pivinv = 1.0/Anew[icol,icol]
        Anew[icol,icol] = 1.0
        Anew[icol,:] *= pivinv
        bnew[icol] *= pivinv
        for ll in np.arange(n):
            if ll != icol:
                dum = Anew[ll,icol]
                Anew[ll,icol] = 0.0
                for l in np.arange(n):
                    Anew[ll,l] -= Anew[icol,l]*dum
                bnew[ll] -= bnew[icol]*dum
            
    for l in np.arange(n-1,-1,-1):
        if indxr[l] != indxc[l]:
            for k in np.arange(n):
                Anew[k,indxr[l]], Anew[k,indxc[l]] = Anew[k,indxc[l]], Anew[k,indxr[l]]
    return Anew,bnew

### Benchmarking Speeds

In [9]:
nrun = 1000 # number of times to run inversion algorithms for benchmarking
print("Time to run LU decomp %d times: %.4f" % (nrun, timeit.timeit(wrapper(LUsolve,A,f), number = nrun)),"s")
print("Time to run GJ (full pivoting) %d times: %.4f" % (nrun, timeit.timeit(wrapper(gaussJordan,np.copy(A),np.copy(f)), number = nrun)),"s")

Time to run LU decomp 1000 times: 0.0633 s
Time to run GJ (full pivoting) 1000 times: 5.7486 s


Theoretically, LU decomposition should be faster than Gauss-Jordan Elimination ($\mathcal{O}(N^3/3)$ versus $\mathcal{O}(N^3/2)$). We do see that LU decomposition runs faster than Gauss-Jordan but the difference is much higher than expected (approximately two orders of magnitude). This is because the LU decomposition function called is a wrapper around LAPACK which is extremely efficient. The Gauss-Jordan code written is a poor adaption from C code to Python translated by myself from "Numerical Recipes" By William Press. With improvements that would be natural when working with numpy arrays such as vectorization I am sure the speed of the Gauss-Jordan algorithm could be increased but it is difficult for any code written in a high level language such as Python to beat a wrapper around C or Fortran code. Hence why one of the greatest uses of Python in scientific computing is to act as a "glue" language. In practice, for any heavy computing it is almost always better to use wrappers of routines written in low level languages.

If we would like to solve for different combinations of floor-mass and pretension loading it would be optimal to use LU decomposition rather than Gauss-Jordan. Not only is there a LAPACK routine available, thus making the computations highly efficient, but Gauss-Jordan is sub-optimal when solving for multiple RHS vectors if they are not known ahead of time. Although Gauss-Jordan will compute some $A^{-1}$, meaning you could multiply a new RHS vector by this inverse, it is somewhat susceptible to round-off error and it is much safer to include this RHS vector during the calculation to begin with. LU decomposition allows one to solve $\mathbf{A} \cdot \mathbf{x} = \mathbf{f}$ for new RHS vectors using only simple forward and back substitution!

However, if memory is of great concern it may be more efficient to use Gauss-Jordan. LU decomposition creates three arrays, P, L, and U, which all have size $N\times N$, thus requiring the user to store $3N^2$ values. Gauss-Jordan converts the array $\mathbf{A}$ into $\mathbf{A}^{-1}$ during its execution, thus making it quite memory efficient.

Now, let us solve for $\mathbf{x}$ using both LU decomposition and Gauss-Jordan to show that they provide equivalent answers (and are correct!)

In [10]:
# Solve using LU
xlu = LUsolve(A,f)
xlu

array([ 20307.10678119,  20307.10678119,  15407.10678119,  15407.10678119,
        10507.10678119,  10507.10678119,   5607.10678119,   5607.10678119,
         1414.21356237,   1414.21356237,   1414.21356237,    707.10678119,
         1000.        ,   1000.        ,   1000.        ,   1000.        ])

In [11]:
# Solve using Gauss-Jordan
Ainv, xgj = gaussJordan(np.copy(A),np.copy(f))
xgj

array([ 20307.10678119,  20307.10678119,  15407.10678119,  15407.10678119,
        10507.10678119,  10507.10678119,   5607.10678119,   5607.10678119,
         1414.21356237,   1414.21356237,   1414.21356237,    707.10678119,
         1000.        ,   1000.        ,   1000.        ,   1000.        ])

In [13]:
# tests whether the two solutions are close (within a tolerance)
# np.isclose(a,b) is equivalent to absolute(a - b) <= (1e-08 + 1e-05 * absolute(b))
np.isclose(xlu,xgj)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

Cool! Our solutions are equivalent (dimensions are in Newtons)... Let us test to see if it is correct (by plugging $\mathbf{x}$ back into $\mathbf{A} \cdot \mathbf{x}$ and checking that it is equal to $\mathbf{f}$)

In [14]:
np.isclose(np.dot(A,xlu), f)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

Now let's solve for three different cases (we will use LU decomposition to do so for the reasons stated previously).

In all cases $d_1=d_3=d_5=d_7=d$.

Case 1) $m=1000 \text{kg}$, $d=1000 \text{N}$

Case 2) $m=4000 \text{kg}$, $d=1000 \text{N}$

Case 3) $m=1000 \text{kg}$, $d=4000 \text{N}$

In [15]:
# Case 1
m = 1000
d1=d3=d5=d7=1000
A,f = initAf(m,d1,d3,d5,d7)
xlu_case1 = LUsolve(A,f)
xlu_case1

array([ 20307.10678119,  20307.10678119,  15407.10678119,  15407.10678119,
        10507.10678119,  10507.10678119,   5607.10678119,   5607.10678119,
         1414.21356237,   1414.21356237,   1414.21356237,    707.10678119,
         1000.        ,   1000.        ,   1000.        ,   1000.        ])

In [16]:
# Case 2
m = 4000
d1=d3=d5=d7=1000
A,f = initAf(m,d1,d3,d5,d7)
xlu_case1 = LUsolve(A,f)
xlu_case1

array([ 79107.10678119,  79107.10678119,  59507.10678119,  59507.10678119,
        39907.10678119,  39907.10678119,  20307.10678119,  20307.10678119,
         1414.21356237,   1414.21356237,   1414.21356237,    707.10678119,
         1000.        ,   1000.        ,   1000.        ,   1000.        ])

In [17]:
# Case 3
m = 1000
d1=d3=d5=d7=4000
A,f = initAf(m,d1,d3,d5,d7)
xlu_case1 = LUsolve(A,f)
xlu_case1

array([ 22428.42712475,  22428.42712475,  17528.42712475,  17528.42712475,
        12628.42712475,  12628.42712475,   7728.42712475,   7728.42712475,
         5656.85424949,   5656.85424949,   5656.85424949,   2828.42712475,
         4000.        ,   4000.        ,   4000.        ,   4000.        ])